In [17]:
!pip uninstall tensorflow
!pip install tensorflow-macos

Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Would remove:
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/import_pb_to_tensorboard
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/saved_model_cli
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/tensorboard
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/tf_upgrade_v2
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/tflite_convert
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/toco
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/toco_from_protos
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/tensorflow-2.16.1.dist-info/*
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/tensorflow/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 3.8 MB/s eta 0:00:0000:0100:02
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
 

In [20]:
!yes | pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cpu/torch_stable.html

Found existing installation: torch 2.4.1
Uninstalling torch-2.4.1:
  Would remove:
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/convert-caffe2-to-onnx
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/convert-onnx-to-caffe2
    /Users/mukhit.ismailov/Work/MLModel/venv/bin/torchrun
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/functorch/*
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/torch-2.4.1.dist-info/*
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/torch/*
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/torchgen/*
Proceed (Y/n)?   Successfully uninstalled torch-2.4.1
Found existing installation: torchvision 0.19.1
Uninstalling torchvision-0.19.1:
  Would remove:
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/torchvision-0.19.1.dist-info/*
    /Users/mukhit.ismailov/Work/MLModel/venv/lib/python3.11/site-packages/torchvision/*
Proceed (Y/n)?   Suc

In [11]:
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:
%%time

import PyPDF2
from transformers import pipeline
import torch
device = 0 if torch.cuda.is_available() else -1


# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

batch_size = 2048
# Function to summarize the extracted text
# def summarize_text(text):
#     summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
#     summary = summarizer(text, max_length=1000, min_length=30, do_sample=False, truncation=True)
#     return summary[0]['summary_text']

def summarize_long_text(text):
    chunks = [text[i:i + batch_size] for i in range(0, len(text), batch_size)]
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    summaries = [summarizer(chunk, max_length=130, min_length=30, do_sample=False, truncation=True)[0]['summary_text'] for chunk in chunks]
    for i, summary in enumerate(summaries):
        print(f"Summary: {summary}\n\n")
    return ' \n\n'.join(summaries)

# Main execution
pdf_path = "/Users/mukhit.ismailov/Work/MLModel/files/hr_policy.pdf"  # Specify your PDF file path
extracted_text = extract_text_from_pdf(pdf_path)
# print(extracted_text)

# Summarize the extracted text
if extracted_text:
    print("--------------------")
    print("--------------------")
    print("--------------------")
    summary_concat = summarize_long_text(extracted_text)
    print(f"Summary: {summary_concat}")
    # summary = summarize_text(summary_concat)
    # print(f"Summary: {summary}")
else:
    print("No text found in the PDF.")



--------------------
--------------------
--------------------


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summary: Our policies apply to all employees, without distinction of position or title, including those under probation, tenured employees and interns. This ensures consistency and transparency and helps to create an enabling culture aligned with organizational beliefs and purpose. You may also share your concerns at myvoice@g42.ai.


Summary: The Salary breakdown we follow is: Basic Salary – 60% , Housing Allowance – 30% , Transportation Allowance - 10%. These three elements form the basis for Bonus calculations. You can access your pay slip through OMNI. Salary is paid monthly before the last day of the month.


Summary: You will be entitled to an annual airfare allowance as mentioned in your offer letter at the time of joining. The allowance will be credited in 12 equal installments, along with your monthly salary. UAE Nationals are entitled to one month’s salary as annual air fare allowance.


Summary: Workmen Compensation Insurance – This is a mandatory insurance coverage to all e

In [12]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def pegasus_summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=30, length_penalty=2.0)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text
# Example usage
pdf_path = "/Users/mukhit.ismailov/Work/MLModel/files/hr_policy.pdf"  # Specify your PDF file path
long_text = extract_text_from_pdf(pdf_path)
summary = pegasus_summarize(long_text)
print(summary)

ImportError: 
PegasusTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
import pytesseract
from PIL import Image
import numpy as np
from pytesseract import Output
import pandas as pd
import pypdfium2 as pdfium


def main():
    s3_bucket = 'vantage-data-sample-tmp'
    src_folder = 'OCR/input/'

    ocr_extract = OcrExtract(s3_bucket, src_folder)
    df = ocr_extract.process_images()
    df = ocr_extract.summarize_text(df)
    api.save(df)


class OcrExtract:

    def __init__(self, s3_bucket, src_folder):
        self.src_folder = src_folder
        self.s3_bucket = s3_bucket

    def convert_pdf2image(self):
        filenames = []
        item_list = api.get_files_from_s3(self.s3_bucket, self.src_folder)
        for item in item_list:
            if str(item.key).endswith('pdf'):
                filename = api.load_file_from_s3(item)
                pdf = pdfium.PdfDocument(filename)
                for i in range(len(pdf)):
                    page = pdf[i]
                    image = page.render(scale=4).to_pil()
                    file_name = filename.split("/")[-1]
                    file_name = f"{file_name}_page{i+1:03d}.jpg"
                    image.save(file_name)
                    filenames.append(file_name)
        return filenames

    def get_filenames(self) -> list:
        try:
            item_list = api.get_files_from_s3(self.s3_bucket, self.src_folder)
            filenames = []
            for item in item_list:
                filename = api.load_file_from_s3(item)
                if not filename.endswith('/') and not filename.endswith('__') and not filename.endswith('pdf'):
                    filenames.append(filename)
            if not filenames:
                LOG.exception(f"Caught an exception during get_filenames() - no files found in {self.src_folder}")
            return filenames
        except Exception as e:
            LOG.exception(f"Caught an exception during get_filenames() - {e}")

    def get_image_data(self, filename) -> pd.DataFrame:
        try:
            return pytesseract.image_to_data(Image.open(filename),lang = 'eng+ara', output_type=Output.DATAFRAME)
        except Exception as e:
            LOG.exception(f"Caught an exception during get_image_data() - {e}")

    def create_image_segment_tesseract(self, image_data, index, text_index, texts) -> dict:
        try:
            xpos = image_data['left'][index]
            ypos = image_data['top'][index]
            width = image_data['width'][index]
            height = image_data['height'][index]
            shape = "RECTANGLE"
            text = texts[text_index]
            coordinates = [xpos, ypos, xpos + width, ypos, xpos + width, ypos + height, xpos, ypos + height]

            return {
                "coordinates": coordinates,
                "shape": shape,
                "object_label": None,
                "text": text
            }
        except Exception as e:
            LOG.exception(f"Caught an exception during create_image_segment_tesseract() - {e}")

    def populate_text(self, image_data, getting_sentences_id) -> list:
        try:
            text_result = []
            for index in getting_sentences_id[0]:
                text_result.append(str(image_data['text'][index]))
            return text_result
        except Exception as e:
            LOG.exception(f"Caught an exception during populate_text() - {e}")


    def process_ocr_boxes(self, image_data) -> list:
        try:
            convert_d = np.asarray(image_data['level'])
            getting_sentences_id = np.where(convert_d == 5)
            texts = self.populate_text(image_data, getting_sentences_id)
            image_segments = []
            for text_index, index in enumerate(getting_sentences_id[0]):
                if len(texts[text_index].strip()) > 0:
                    image_segments.append(self.create_image_segment_tesseract(image_data, index, text_index, texts))
            return image_segments, texts
        except Exception as e:
            LOG.exception(f"Caught an exception during process_ocr_boxes() - {e}")


    def process_images(self, return_with_boxes=False) -> pd.DataFrame:
        try:
            filenames_pdf = self.convert_pdf2image()
            filenames_image = self.get_filenames()
            filenames = filenames_pdf + filenames_image
            data = []  # Initialize an empty list
            for filename in filenames:
                file_name = filename.split("/")[-1]
                image_data = self.get_image_data(filename)
                output, text_output = self.process_ocr_boxes(image_data)
                print(f"----{file_name}----")
                print(" ".join(text_output))
                # Determine if the file is a PDF or an image based on the extension
                pdf_or_image = 'pdf' if ".pdf" in filename else 'image'
                # Create a dictionary and append it to the list
                data.append({
                    'filename': file_name,
                    'pdf_or_image': pdf_or_image,
                    'text_output': " ".join(text_output)
                })
            # Convert the list of dictionaries into a DataFrame
            df_combined = pd.DataFrame(data)
            return df_combined
        except Exception as e:
            LOG.exception(f"Caught an exception during process_images() - {e}")

    def summarize_text(self, df):
        df['text_summarized'] = df['text_output'].apply(self.summarize_long_text)
        return df

    def summarize_long_text(self, text):
        import torch
        from transformers import pipeline
        batch_size = 2048
        device = 0 if torch.cuda.is_available() else -1
        chunks = [text[i:i + batch_size] for i in range(0, len(text), batch_size)]
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
        summaries = [summarizer(chunk, max_length=130, min_length=30, do_sample=False, truncation=True)[0]['summary_text'] for chunk in chunks]
        for i, summary in enumerate(summaries):
            print(f"Summary: {summary}\n\n")
        return ' \n\n'.join(summaries)


main()


In [ ]:
# import pytesseract
# from PIL import Image
# import numpy as np
# from pytesseract import Output
# import pandas as pd
# import pypdfium2 as pdfium
#
#
# def main():
#     s3_bucket = 'vantage-data-sample-tmp'
#     src_folder = 'OCR/input/'
#     tgt_folder = 'OCR/output/'
#
#     ocr_extract = OcrExtract(s3_bucket, src_folder, tgt_folder)
#     ocr_extract.process_images(s3_bucket, src_folder, tgt_folder)
#
#
# class OcrExtract:
#     '''
#     Processes images and extracts english and arabic texts into a dataframe.
#     '''
#     def __init__(self, s3_bucket, src_folder, tgt_folder):
#         self.src_folder = src_folder
#         self.tgt_folder = tgt_folder
#         self.s3_bucket = s3_bucket
#
#     # def convert_pdf2image(self):
#
#     def convert_pdf2image(self,s3_bucket,src_folder):
#         '''
#         Finds PDF files in s3 path, converts them to JPG images and uploads back to s3 path.
#
#         :param s3_bucket (str): s3 bucket name
#         :param src_folder (str): s3 path
#         :return: None
#         '''
#         try:
#             item_list = api.get_files_from_s3(s3_bucket, src_folder)
#             for item in item_list:
#                 if str(item.key).endswith('pdf'):
#                     filename = api.load_file_from_s3(item)
#                     pdf = pdfium.PdfDocument(filename)
#                     for i in range(len(pdf)):
#                         page = pdf[i]
#                         image = page.render(scale=4).to_pil()
#                         file_name = filename.split("/")[-1]
#                         image.save(f"{file_name}_{i:03d}.jpg")
#                         api.load_file_to_s3(f"{file_name}_{i:03d}.jpg", s3_bucket, f"{src_folder}{file_name}_{i:03d}.jpg")
#         except Exception as e:
#             LOG.exception(f"Caught an exception during convert_pdf2image() - {e}")
#
#     def get_filenames(self, s3_bucket, src_folder) -> list:
#         '''
#         Method to load files available on s3 path.
#
#         :param s3_bucket (str): s3 bucket name
#         :param src_folder (str): s3 path
#         :return: list of file names
#         '''
#         try:
#             item_list = api.get_files_from_s3(s3_bucket, src_folder)
#             filenames = []
#             for item in item_list:
#                 filename = api.load_file_from_s3(item)
#                 if not filename.endswith('/') and not filename.endswith('__') and not filename.endswith('pdf'):
#                     filenames.append(filename)
#             if not filenames:
#                 LOG.exception(f"Caught an exception during get_filenames() - no files found in {src_folder}")
#             return filenames
#         except Exception as e:
#             LOG.exception(f"Caught an exception during get_filenames() - {e}")
#
#     def get_image_data(self, filename) -> pd.DataFrame:
#         '''
#         Method to extract english and arabic text with coordinates and shape details from images into pandas dataframe.
#
#         :param filename (str): file name
#         :return: dataframe
#         '''
#         try:
#             return pytesseract.image_to_data(Image.open(filename),lang = 'eng+ara', output_type=Output.DATAFRAME)
#         except Exception as e:
#             LOG.exception(f"Caught an exception during get_image_data() - {e}")
#
#     def create_image_segment_tesseract(self, image_data, index, text_index, texts) -> dict:
#         '''
#         Method to extract coordinates and shape information from output dataframe.
#
#         :param image_data (df): dataframe with extracted information from the image
#         :param index (int): index value of the row that contains text value
#         :param text_index (int): index value of the row of the getting_sentences_id[0]
#         :param texts (list): list of all text values from the image
#         :return: dictionary with coordinates and shape details from the image
#         '''
#         try:
#             xpos = image_data['left'][index]
#             ypos = image_data['top'][index]
#             width = image_data['width'][index]
#             height = image_data['height'][index]
#             shape = "RECTANGLE"
#             text = texts[text_index]
#             coordinates = [xpos, ypos, xpos + width, ypos, xpos + width, ypos + height, xpos, ypos + height]
#
#             return {
#                 "coordinates": coordinates,
#                 "shape": shape,
#                 "object_label": None,
#                 "text": text
#             }
#         except Exception as e:
#             LOG.exception(f"Caught an exception during create_image_segment_tesseract() - {e}")
#
#     def populate_text(self, image_data, getting_sentences_id) -> list:
#         '''
#         Method to extract content from the rows that were identified as having text.
#
#         :param image_data (df): dataframe with extracted information from the image
#         :param getting_sentences_id (array): numpy array with index values of rows containing text content
#         :return: list with text values from the image
#         '''
#         try:
#             text_result = []
#             for index in getting_sentences_id[0]:
#                 text_result.append(str(image_data['text'][index]))
#             return text_result
#         except Exception as e:
#             LOG.exception(f"Caught an exception during populate_text() - {e}")
#
#
#     def process_ocr_boxes(self, image_data) -> list:
#         '''
#         Method to process images, extract rows with text, cooridnates and shape details into a list.
#
#         :param image_data (df): dataframe with extracted information from the image
#         :return: list with image segments and list with text values from the image
#         '''
#         try:
#             convert_d = np.asarray(image_data['level'])
#             getting_sentences_id = np.where(convert_d == 5)
#             texts = self.populate_text(image_data, getting_sentences_id)
#             image_segments = []
#             for text_index, index in enumerate(getting_sentences_id[0]):
#                 if len(texts[text_index].strip()) > 0:
#                     image_segments.append(self.create_image_segment_tesseract(image_data, index, text_index, texts))
#             return image_segments, texts
#         except Exception as e:
#             LOG.exception(f"Caught an exception during process_ocr_boxes() - {e}")
#
#
#     def save_output(self, s3_bucket, tgt_folder, df, file_name):
#         '''
#         Optional method to save output df as a csv on s3 location. Can be used in case each image output needs to be saved separately.
#
#         :param s3_bucket (str): s3 bucket name
#         :param tgt_folder (str): target s3 path
#         :param df (df): dataframe to be saved
#         :param file_name (str): output file name
#         :return: None
#         '''
#         try:
#             df.to_csv(f"{file_name}.csv", encoding='utf-8', index=False)
#             api.load_file_to_s3(f"{file_name}.csv", s3_bucket, f"{tgt_folder}{file_name}.csv")
#         except Exception as e:
#             LOG.exception(f"Caught an exception during save_output() - {e}")
#
#     def save_output_df(self, df):
#         '''
#         Method to save output df as a parquet file on s3 location. The df will contain output combined from all images and saved as one parquet file.
#
#         :param df (df): dataframe to be saved
#         :return: None
#         '''
#         try:
#             api.save(df)
#         except Exception as e:
#             LOG.exception(f"Caught an exception during save_output_df() - {e}")
#
#
#     def process_images(self, s3_bucket, src_folder, tgt_folder):
#         '''
#         Main method to process images.
#
#         :param s3_bucket (str): s3 bucket name
#         :param src_folder (str): input s3 path
#         :param tgt_folder (str): target s3 path
#         :return None
#         '''
#         try:
#             self.convert_pdf2image(s3_bucket,src_folder)
#             filenames = self.get_filenames(s3_bucket, src_folder)
#             df_combined = pd.DataFrame()
#             for filename in filenames:
#                 file_name = filename.split("/")[-1]
#                 image_data = self.get_image_data(filename)
#                 output, text_output = self.process_ocr_boxes(image_data)
#                 print(f"----{file_name}----")
#                 print(" ".join(text_output))
#                 df = pd.DataFrame(output)
#                 df['filename'] = file_name
#                 for col in df.columns:
#                     df[col] = df[col].apply(lambda s: str(s)).astype('str')
#                 # self.save_output(s3_bucket, tgt_folder, df, file_name)
#                 df_combined = pd.concat([df_combined, df])
#             self.save_output_df(df_combined)
#         except Exception as e:
#             LOG.exception(f"Caught an exception during process_images() - {e}")
#
# # define s3 bucket, and input and output paths
# main()


In [20]:
%%time
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
# device = -1
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))
# >>> [{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]
CPU times: user 19.7 s, sys: 4.86 s, total: 24.6 s
Wall time: 8.9 s
